In [4]:
import matplotlib
import matplotlib.path as mplPath
import numpy as np
import matplotlib.pyplot as plt
import gmplot
import itertools
import requests
import copy
import re
import json
import csv
from IPython.core.display import display
from IPython.lib.display import IFrame
import os
%matplotlib inline
#%matplotlib auto


In [14]:
x = """		label: "TEST_1_MarkerItIs",
		icon: img,"""
for match in re.finditer(r'MarkerItIs[",\n\s\t]+icon:\s(img)', x, re.S):
    span = match.span(1)
    print span
    x = x[:span[0]] + '" "' + x[span[1]:]
print x
#print re.search(r'MarkerItIs[",\n\s\t]+icon:\s(img)', x).span()[1]
#(?<=BookTitle).*?(?=\s)

(38, 41)
		label: "TEST_1_MarkerItIs",
		icon: " ",


## Start here

In [15]:
# прорисовка и проверка на коллижины
# TODO: коллижины не по точкам, а по областям, а так же парсинг изоскопа
class PostMatcher(object):
    def __init__(self, points, all_postals = [], time = 9000):
        if not type(points) is list or not type(points[0]) is dict:
            # [float] in case we will change from 1 point to set of points
            print "points should be a list of objects: [{postal: *postalcode*, x: *[float]*, y: *[float]*}]"
            raise Exception(points)
        self.points = points
        self.all_postals = all_postals
        self.time_space = []
        self.time = time
        
    def to_array(self, arr):
        x, y = [], []
        for i in arr:
            d = i.split(',')
            x.append(float(d[0]))
            y.append(float(d[1]))
        return x, y
            
    # thx and soz http://www.flаvіоgоrtаnа.com/isоscоpe/ - don't copy it, there're cyryllic letters
    def getSpace(self, postal, x, y, time): # time in seconds
        # gonna impelement parsing here
        arr = ["59.8720551,10.7900333","59.8720551,10.7913208","59.8722267,10.7918358","59.87257,10.7921791","59.8729134,10.7932091","59.8732567,10.7935524","59.8734283,10.7940674","59.8734283,10.794754","59.8736,10.795269","59.8742867,10.7959557","59.87463,10.7959557","59.8749733,10.7956123","59.8754883,10.7954407","59.8761749,10.7954407","59.8766899,10.7956123","59.8773766,10.796299","59.8775482,10.796814","59.8775482,10.7975006","59.8777199,10.7980156","59.8780632,10.7980156","59.8782349,10.7975006","59.8782349,10.7961273","59.8784065,10.7956123","59.8797798,10.794239","59.8801231,10.794239","59.8814964,10.7956123","59.8816681,10.7961273","59.8816681,10.7975006","59.8818398,10.7980156","59.8821831,10.7983589","59.8823547,10.7988739","59.8823547,10.7995605","59.8825264,10.8000755","59.8835564,10.8004189","59.8838997,10.8007622","59.8844147,10.8009338","59.8849297,10.8007622","59.8859596,10.7997322","59.8864746,10.7995605","59.8871613,10.7995605","59.8876762,10.7997322","59.8880196,10.8007622","59.8883629,10.8011055","59.8885345,10.8016205","59.8885345,10.8023071","59.8887062,10.8028221","59.8890495,10.8028221","59.8893929,10.8024788","59.8904228,10.8021355","59.8907661,10.8017921","59.8912811,10.8016205","59.8919678,10.8016205","59.8924828,10.8014488","59.8928261,10.8011055","59.8933411,10.8009338","59.893856,10.8011055","59.8941994,10.8014488","59.8947144,10.8016205","59.895401,10.8016205","59.895916,10.8017921","59.8966026,10.8024788","59.8967743,10.8029938","59.8967743,10.8050537","59.896946,10.8055687","59.8972893,10.805912","59.8974609,10.806427","59.8974609,10.8078003","59.8976326,10.8083153","59.8979759,10.8086586","59.8983192,10.8096886","59.8986626,10.8100319","59.8990059,10.8110619","59.8993492,10.8114052","59.8995209,10.8119202","59.8995209,10.8139801","59.8993492,10.8144951","59.8990059,10.8148384","59.8986626,10.8158684","59.8981476,10.81604","59.8974609,10.81604","59.896946,10.8158684","59.8966026,10.8148384","59.8962593,10.8144951","59.895916,10.8134651","59.8955727,10.8131218","59.8952293,10.8120918","59.894886,10.8117485","59.8947144,10.8112335","59.8947144,10.8098602","59.8945427,10.8093452","59.8940277,10.8091736","59.8935127,10.8093452","59.8928261,10.8100319","59.8926544,10.8105469","59.8926544,10.8112335","59.8924828,10.8117485","59.8921394,10.8120918","59.8917961,10.8131218","59.8912811,10.8132935","59.8905945,10.8132935","59.8900795,10.8134651","59.8890495,10.8144951","59.8885345,10.8146667","59.8878479,10.8146667","59.8873329,10.8144951","59.8869896,10.8134651","59.8866463,10.8134651","59.8863029,10.8144951","59.8859596,10.8148384","59.885788,10.8153534","59.885788,10.81604","59.8859596,10.816555","59.8863029,10.8168983","59.8864746,10.8174133","59.8864746,10.8194733","59.8863029,10.8199883","59.8859596,10.8203316","59.885788,10.8208466","59.885788,10.8229065","59.8856163,10.8234215","59.885273,10.8237648","59.8851013,10.8242798","59.8851013,10.8249664","59.8849297,10.8254814","59.8845863,10.8258247","59.884243,10.8268547","59.883213,10.8278847","59.8830414,10.8283997","59.8830414,10.8290863","59.8828697,10.8296013","59.8825264,10.8299446","59.8823547,10.8304596","59.8825264,10.8309746","59.8845863,10.8330345","59.8851013,10.8332062","59.8856163,10.8330345","59.8859596,10.8326912","59.8864746,10.8325195","59.8885345,10.8325195","59.8890495,10.8323479","59.8893929,10.8320045","59.8899078,10.8318329","59.8905945,10.8318329","59.8911095,10.8320045","59.8914528,10.8330345","59.8917961,10.8333778","59.8919678,10.8338928","59.8919678,10.8345795","59.8917961,10.8350945","59.8914528,10.8354378","59.8912811,10.8359528","59.8912811,10.8366394","59.8911095,10.8371544","59.8900795,10.8381844","59.8897362,10.8392143","59.8890495,10.839901","59.8885345,10.8400726","59.8880196,10.839901","59.8869896,10.838871","59.8864746,10.8386993","59.885788,10.8386993","59.885273,10.838871","59.8845863,10.8395576","59.884243,10.8405876","59.883213,10.8409309","59.8830414,10.8414459","59.8830414,10.8421326","59.8828697,10.8426476","59.8818398,10.8429909","59.8814964,10.8433342","59.8809814,10.8435059","59.8796082,10.8435059","59.8790932,10.8433342","59.8784065,10.8426476","59.8782349,10.8421326","59.8782349,10.8366394","59.8784065,10.8361244","59.8790932,10.8354378","59.8801231,10.8350945","59.8801231,10.8347511","59.8797798,10.8344078","59.8796082,10.8338928","59.8796082,10.8332062","59.8794365,10.8326912","59.8790932,10.8326912","59.8784065,10.8333778","59.8780632,10.8344078","59.8775482,10.8345795","59.8768616,10.8345795","59.8763466,10.8344078","59.8756599,10.8337212","59.8754883,10.8332062","59.8754883,10.8325195","59.8753166,10.8320045","59.8749733,10.8320045","59.8748016,10.8325195","59.8748016,10.8345795","59.87463,10.8350945","59.8742867,10.8354378","59.8739433,10.8364677","59.8734283,10.8366394","59.8727417,10.8366394","59.8722267,10.8364677","59.8718834,10.8361244","59.8715401,10.8361244","59.8708534,10.8368111","59.8706818,10.837326","59.8706818,10.8380127","59.8705101,10.8385277","59.8699951,10.8386993","59.8693085,10.8386993","59.8687935,10.8385277","59.8681068,10.837841","59.8679352,10.837326","59.8679352,10.8345795","59.8677635,10.8338928","59.8672485,10.8333778","59.8665619,10.8332062","59.8658752,10.8332062","59.8653603,10.8333778","59.8633003,10.8354378","59.8631287,10.8359528","59.8631287,10.837326","59.862957,10.837841","59.862442,10.8380127","59.8610687,10.8380127","59.8605537,10.837841","59.8603821,10.837326","59.8603821,10.8345795","59.8602104,10.8340645","59.8596954,10.8338928","59.8590088,10.8338928","59.8584938,10.8340645","59.8581505,10.8344078","59.8571205,10.8347511","59.8567772,10.8350945","59.8557472,10.8354378","59.8554039,10.8357811","59.8543739,10.8361244","59.8540306,10.8364677","59.8530006,10.8368111","59.8526573,10.8371544","59.8516273,10.8374977","59.8505974,10.8385277","59.8495674,10.838871","59.8492241,10.8392143","59.8487091,10.839386","59.8481941,10.8392143","59.8475075,10.8385277","59.8473358,10.8380127","59.8473358,10.8345795","59.8475075,10.8340645","59.8485374,10.8330345","59.8487091,10.8325195","59.8487091,10.8290863","59.8488808,10.8285713","59.8493958,10.8283997","59.8500824,10.8283997","59.8505974,10.8285713","59.851284,10.829258","59.8514557,10.8297729","59.8514557,10.8304596","59.8516273,10.8309746","59.8521423,10.8311462","59.8526573,10.8309746","59.8530006,10.8306313","59.8540306,10.8302879","59.8543739,10.8299446","59.8554039,10.8296013","59.8557472,10.829258","59.8567772,10.8289146","59.8571205,10.8285713","59.8576355,10.8283997","59.8583221,10.8283997","59.8588371,10.828228","59.8590088,10.827713","59.8590088,10.8263397","59.8591805,10.8258247","59.8602104,10.8247948","59.8603821,10.8242798","59.8603821,10.8229065","59.8602104,10.8223915","59.8598671,10.8220482","59.8596954,10.8215332","59.8596954,10.8208466","59.8595238,10.8203316","59.8591805,10.8199883","59.8590088,10.8194733","59.8591805,10.8189583","59.8595238,10.818615","59.8596954,10.8181","59.8596954,10.8174133","59.8595238,10.8168983","59.8591805,10.816555","59.8590088,10.81604","59.8590088,10.8119202","59.8591805,10.8114052","59.8598671,10.8107185","59.8603821,10.8105469","59.8608971,10.8107185","59.8615837,10.8114052","59.861927,10.8124352","59.8622704,10.8124352","59.862442,10.8119202","59.862442,10.8078003","59.8626137,10.8072853","59.8636436,10.8062553","59.8638153,10.8057404","59.8638153,10.8036804","59.863987,10.8031654","59.8646736,10.8024788","59.8657036,10.8021355","59.8660469,10.8011055","59.8670769,10.8000755","59.8672485,10.7995605","59.8672485,10.796814","59.8674202,10.796299","59.8677635,10.7959557","59.8681068,10.7949257","59.8691368,10.7945824","59.8693085,10.7940674","59.8693085,10.7899475","59.8694801,10.7894325","59.8699951,10.7892609","59.8713684,10.7892609","59.8718834,10.7893467","59.8720551,10.7896042","59.8720551,10.7900333"]
        lat, lon = self.to_array(arr)
        return {'postal': postal, 'x': x, 'y': y, 'dest_lat': lat, 'dest_lon': lon}
                
    # checking whether there's collision
    # element: {postal: string, dest_lat: [float], dest_lon: [float]}
    # check_with: [{postal: string, x: float, y: float}]
    def check_collision(self, element, check_with):
        result = {'postal': element['postal'], 'intersects': [], 'nonintersect': []}
        # getting our path Area
        path_data = []
        path_data.append((mplPath.Path.MOVETO, (element['dest_lat'][0], element['dest_lon'][1])))
        for x, y in itertools.izip(element['dest_lat'][1:], element['dest_lon'][1:]):
            path_data.append((mplPath.Path.LINETO, (x, y)))
        # closing our rectangle
        path_data.append((mplPath.Path.LINETO, (element['dest_lat'][0], element['dest_lon'][0])))
        # formating it for matplotlib.Path
        codes, verts = zip(*path_data)
        path = mplPath.Path(verts, codes)
        # checking intersections
        for i, val in enumerate(check_with):
            if path.contains_point((val['x'], val['y'])):
                # should add check for dublicates
                result['intersects'].append(val)
            else:
                result['nonintersect'].append(val)
        return result
            #self, areas, elements, markers = [], **kwargs
            #self, offices, postal_areas, **kwargs
    def draw(self, offices, postal_areas, **kwargs):
        fileName = kwargs.get('fileName',"map.html")
        
        # getting list of usable postals
        show_used = kwargs.get('show_used', False)        
        usedPostals = list(itertools.chain.from_iterable([x['postal_collision'] for x in offices])) 
        # list of unique elements
        seen = set()
        seen_add = seen.add
        usedPostals = [x for x in usedPostals if not (x in seen or seen_add(x))]
        if not show_used:
            # saving only needed postal_areas
            elements = copy.deepcopy(postal_areas)
            postal_areas = []
            for el in elements:
                if el['postal'] in usedPostals:
                    postal_areas.append(el)

        # getting "focusing point" - where map will focus and initiating map view
        x = y = None
        #for currEl, nextEl in zip(the_list, the_list[1:]):
        for area in postal_areas:
            avgX = np.average(area['dest_lat'])
            avgY = np.average(area['dest_lon'])
            if x is None:
                x, y = avgX, avgY
            else:
                x, y = np.average([x, avgX]), np.average([y, avgY])
                
        zoomLevel = kwargs.get('zoomLevel', 12)
        print x, y
        gmap = gmplot.GoogleMapPlotter(x, y, zoomLevel)
        
        unUsedColor = 'cornflowerblue'
        usedColor = 'forestgreen'
        officeColors = 'khaki'
        
        # drawing postal code zones
        for area in postal_areas:
            # plot || polygon
            if area['postal'] in usedPostals:
                gmap.plot(area['dest_lat'], area['dest_lon'], usedColor, edge_width=3)
                gmap.marker(area['centerX'], area['centerY'], title=area['postal'])
            elif show_used == True:
                gmap.plot(area['dest_lat'], area['dest_lon'], unUsedColor, edge_width=3)
                gmap.marker(area['centerX'], area['centerY'], title=area['postal'])
        
        # in case we draw "taken zone", currently we don't
        for office in offices:
            # plot || polygon
            if 'dest_lat' in office:
                gmap.polygon(office['dest_lat'], office['dest_lon'], officeColors, edge_width=3)
            gmap.marker(np.average(office['centerX']), np.average(office['centerY']), title=office['name'] + '_MarkerItIs')
            
            
        gmap.draw(fileName)
        # проблема в правах доступа и маркер не подтягивает(он находится в папке с библиотекой, а Chrome не дает к ней доступ)
        data = []
        with open(fileName, 'r') as f:
            #https://developers.google.com/maps/documentation/javascript/infowindows
            data = f.read()\
                .replace("<head>", """<head>
                <style>
                    .labels{
                        color: green
                    }
                </style>""")\
                .replace("icon: img,", 'icon: img,\nlabelClass: "labels",').replace("title: ", 'label: ')
            # replacing only for markers(take care, replace length should be equals to replacement length)
            for match in re.finditer(r'_MarkerItIs[",\n\s\t]+icon:\s(img)', data, re.S):
                span = match.span(1)
                data = data[:span[0]] + '0/1' + data[span[1]:] # 0/1 - deserves an award "best line of a code in my life -_-"
            #re.search(r'MarkerItIs[",\n\s\t]+icon:\s(img)', x).span()[1]
        os.remove(fileName)
        with open(fileName, 'a') as f:
            f.write(data)
        display(IFrame(fileName, height = 500, width = "90%"))


    def renderSingle(self, postal, x, y, time, re_render = False):
        # don't use that func often, plx, or use it only when you're SURE (it's parsing site)
        for i, val in enumerate(self.time_space):
            if val['postal'] == postal:
                if not re_render: 
                    break
                self.time_space[i] = self.getSpace(postal, x, y, time)
                break
            else:
                self.time_space.append(self.getSpace(postal, x, y, time))
                
        if len(self.time_space) == 0:
            self.time_space.append(self.getSpace(postal, x, y, time))
            
        return 0
    
    def renderAll(self, re_render = False): # time = 2.5 hours
        for i in self.points:
            self.renderSingle(i['postal'], i['x'][0], i['y'][0], self.time, re_render)
        return self.time_space

In [16]:
# parsing our data points
def getPostalZones():
    postalZones = []
    with open('geocsv_rigth.csv', 'rb') as csvfile:
        spamreader = csv.reader(csvfile, delimiter=',', quotechar='"')
        for row in spamreader:
            if(spamreader.line_num == 1):
                continue
            coords = row[0].replace('POLYGON ', '').replace('(', '').replace(")", '').split(',')
            x, y = [], []
            for el in coords:
                el = el.split(' ')
                x.append(float(el[-1]))
                y.append(float(el[0]))
            point = {
                'postal': row[-2],
                'dest_lat': x,
                'dest_lon': y
            }
            postalZones.append(point)
    return postalZones

In [17]:
# эти функции нужно будет закинуь в класс
def readUrl(url):
    headers = {
        'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1)'\
            + ' AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'
    }
    page = requests.get(url, headers=headers)
    return page.content

def location(postcode):
    url = "http://maps.googleapis.com/maps/api/geocode/json?components=country:NO%7Cpostal_code:"\
          + postcode\
          + "&sensor=false"
    print url
    response = readUrl(url)
    data = json.loads(response)
    bounds = data['results'][0]["geometry"]['bounds']
    # creating Rhombus
    latitudes =  [
        (bounds['northeast']['lat'] + bounds['southwest']['lat'])/2,
        bounds['northeast']['lat'],
        (bounds['northeast']['lat'] + bounds['southwest']['lat'])/2,
        bounds['southwest']['lat']
    ]
    longitudes =  [
        bounds['southwest']['lng'],
        (bounds['northeast']['lng'] + bounds['southwest']['lng'])/2,
        bounds['northeast']['lng'],
        (bounds['northeast']['lng'] + bounds['southwest']['lng'])/2
    ]
   
    return latitudes, longitudes

# testing
# нужно бы ещё парсинг добавить и будет гуд
#latitudes = 59.87349399999999
#longitudes = 10.8140581
latitudes, longitudes = location("3684")
#####################################################################
# должен использоваться для выкачки с изоскоупа, сейчас не работает(принимал точку с latitudes, longitudes(центральную))
# как вариант в будущем просто изменить на точки от зайца)
#####################################################################
points = [{
        'postal' : 'TEST',
        'x' : latitudes,
        'y' : longitudes
    }]
check_with = [{
        'postal' : 'COLLISION_TRUE',
        'x': latitudes[0],
        'y': longitudes[0]
    },{
        'postal' : 'COLLISION_FALSE',
        'x': latitudes[0] + 50,
        'y': longitudes[0] + 50
    }]
postalZones = getPostalZones()

http://maps.googleapis.com/maps/api/geocode/json?components=country:NO%7Cpostal_code:3684&sensor=false


In [18]:
# DRAWING
test_single = PostMatcher(points)
test_single.renderAll()

# проверяем есть ли эти точки попадают в нашу зону.
# Это нужно ещё поправить

collisions_test = test_single.check_collision(test_single.time_space[0], check_with)

# рисуем
#################
# тут первые 2 элемента ключевые:
"""
1)test_single.time_space[0:1] - берем первый элемент(желтный), который и есть наша "зона покрытия"(в данный момент всего 1 элем)
2)postalZones[0:1] - тут мы рисуем постал кода для изменения к-ва изменяй [0:1] є ([0:1] : [0:-1]), ну или просто postalZones
"""
def readOffices(fileName):
    postalZones = []
    centralLines = []
    spamreader = csv.reader(csvfile, delimiter=',', quotechar='"')
        for row in spamreader:
            if(spamreader.line_num == 1):
                continue
            coords = row[0].replace('POLYGON ', '').replace('(', '').replace(")", '').split(',')
            x, y = [], []
            for el in coords:
                el = el.split(' ')
                x.append(float(el[-1]))
                y.append(float(el[0]))
            point = {
                'postal': row[-2],
                'dest_lat': x,
                'dest_lon': y
            }
            postalZones.append(point)
    return []
check_postals = [8300,8310,8313,8315,8316,8317,8323,8324,8325,8326,8357,8360,8370,8372,8380,8382,8384,8387,8390,8392]
check_postals = [str(x) for x in check_postals]
offices = [{
        'name': 'TEST_1',
        'centerX': 68.147052,
        'centerY': 13.61052,
        'postal_collision': check_postals
}]
#################[0:5]
# change show_used=False -> show_used=True to show all
test_single.draw(offices, postalZones, show_used=False, fileName='mymap.html')

# sometimes image just won't show - experiment with file name: change it to random and try again
# if that don't work try it over and over again)
# error(f12->console): 
# Google Maps API error: MissingKeyMapError https://developers.google.com/maps/documentation/javascript/error-messages#missing-key-map-error_.jb @ js?libraries=visualization&sensor=true_or_false:34


68.2950841093 14.4218334146


KeyError: 'centerX'

In [52]:
np.average(postalZones[0]['dest_lon'])

10.411304992356081

In [13]:
postalZones[0]

{'centerX': 243221.45,
 'centerY': 6652594.56,
 'dest_lat': [59.9517774411771,
  59.9514851634642,
  59.9503511193262,
  59.9496773309273,
  59.9499433979954,
  59.9508219220884,
  59.9513722684504,
  59.9514245207584,
  59.9514014530553,
  59.9513510837538,
  59.9514873866365,
  59.9577021076413,
  59.9618290613601,
  59.9641450569213,
  59.9678339395045,
  59.9696090673053,
  59.9712410459639,
  59.9715002126325,
  59.9722361177143,
  59.9726789671828,
  59.9728105638736,
  59.9734164728201,
  59.9735167644614,
  59.9738252986657,
  59.9752095082865,
  59.9754795579236,
  59.9758347347018,
  59.9761243174943,
  59.9764417822584,
  59.9764846792789,
  59.9765819745577,
  59.9769033514098,
  59.9770770513345,
  59.9776590062876,
  59.9777977837318,
  59.9780058654313,
  59.9785437411687,
  59.9786524023054,
  59.9788942516388,
  59.9792604465973,
  59.9794684242577,
  59.9795809218206,
  59.9795361674332,
  59.9796168071593,
  59.9795456936337,
  59.9796512510899,
  59.9797394810129,
 